In [2]:
## Tagging entities with <e1> and <e2> tags to prepare for BERT input

In [1]:
import pandas as pd
import numpy as np
import json
import pickle

In [2]:
def change_entities_chem(sentence, diff_chem_1, diff_chem_2, diff_pro_1, diff_pro_2):
    
    first_part = sentence[:diff_chem_1] + ' <e1> ' + sentence[diff_chem_1:diff_chem_2] + ' </e1> ' + sentence[diff_chem_2:diff_pro_1]
    last_part = ' <e2> ' + sentence[diff_pro_1 :diff_pro_2] + ' </e2> ' + sentence[diff_pro_2:]
    
    return first_part + last_part


def change_entities_pro(sentence, diff_chem_1, diff_chem_2, diff_pro_1, diff_pro_2):
    
    first_part = sentence[:diff_chem_1] + ' <e2> ' + sentence[diff_chem_1:diff_chem_2] + ' </e2> ' + sentence[diff_chem_2:diff_pro_1]
    last_part = ' <e1> ' + sentence[diff_pro_1 :diff_pro_2] + ' </e1> ' + sentence[diff_pro_2:]
    
    return first_part + last_part


def change_entities_nested(sentence, diff_chem_1, diff_chem_2, diff_pro_1, diff_pro_2):
    
    if diff_chem_2 < diff_pro_2:
        first_part = sentence[:diff_chem_1] + ' <e1> <e2> ' + sentence[diff_chem_1:diff_chem_2] + ' </e1> ' + sentence[diff_chem_2 :diff_pro_2 ] + ' </e2> ' + sentence[diff_pro_2:] 
        
    else:    
        first_part = sentence[:diff_pro_1] + ' <e2> <e1> ' + sentence[diff_pro_1:diff_pro_2] + ' </e2> ' + sentence[diff_pro_2:diff_chem_2] + ' </e1> ' + sentence[diff_chem_2:] 
        
    return first_part 

In [3]:
def add_tags(file = "../Data/Abstract_Entities/pairs.json"):
    df_sentences = pd.read_json(file)
        
    df_sentences.loc[df_sentences['e1_start'] <  df_sentences['e2_start'], 'changed_sent'] = df_sentences[['e1_start', 'e1_end', 'e2_start', 'e2_end', 'sentence']].apply(lambda x: change_entities_chem(x.sentence, x.e1_start, x.e1_end, x.e2_start, x.e2_end), axis=1)
    df_sentences.loc[df_sentences['e1_start'] >  df_sentences['e2_start'], 'changed_sent'] = df_sentences[['e1_start', 'e1_end', 'e2_start', 'e2_end', 'sentence']].apply(lambda x: change_entities_pro(x.sentence, x.e2_start, x.e2_end, x.e1_start, x.e1_end), axis=1)
    df_sentences.loc[df_sentences['e1_start'] ==  df_sentences['e2_start'], 'changed_sent'] = df_sentences[['e1_start', 'e1_end', 'e2_start', 'e2_end', 'sentence']].apply(lambda x: change_entities_nested(x.sentence, x.e1_start, x.e1_end, x.e2_start, x.e2_end), axis=1)
    

    return df_sentences

In [4]:
df = add_tags()

In [10]:
import random
idx = random.randint(0, len(df))
print(df.iloc[idx]["sentence"])
print(df.iloc[idx]["e1"])
print(df.iloc[idx]["e2"])
print(df.iloc[idx]["changed_sent"])

2.3 Blood routine: white blood cells, neutrophils, lymphocytes, monocytes, eosinophils, basophils, neutrophils, lymphocytes, monocytes, eosinophils Acidic granulocyte count, basophil count, red blood cell, hemoglobin, hematocrit, average volume of red blood cells, average red blood cell Hb content, average red blood cell Hb concentration, RDW standard deviation, RDW coefficient of variation, platelet count, platelet specific platelet average Volume, platelet distribution width,% of large platelets; 2.4 Liver and kidney function tests: alanine aminotransferase, aspartate aminotransferase, alkaline phosphatase, Î³-glutamyl transferase, prealbumin, total protein, albumin, globulin, white / globule ratio , Total bilirubin, direct bilirubin, cholinesterase, urea, creatinine, total carbon dioxide, uric acid glucose, potassium, sodium, chlorine, calcium, corrected calcium, magnesium, phosphorus, calcium and phosphorus product, anion gap, penetration Pressure, total cholesterol, triacylglycero

In [11]:
df.to_json("../Data/Abstract_Entities/pairs_tagger.json")